In [ ]:
!pip install pytube

In [ ]:
from pytube import YouTube
from pydub import AudioSegment

def download_audio_preview(video_url):
    yt = YouTube(video_url)
    audio_stream = yt.streams.filter(only_audio=True).first()

    audio_file = audio_stream.download(filename='audio.mp4')

    audio = AudioSegment.from_file(audio_file)

    preview_audio = audio[:30000]

    preview_audio.export('preview_audio.mp3', format='mp3')
    print("Preview saved as preview_audio.mp3")
video_url = 'https://www.youtube.com/watch?v=hggjZ-FGGkw'
download_audio_preview(video_url)


In [ ]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
print("Connected!")
db = client['audio']

In [ ]:
import gridfs

fs = gridfs.GridFS(db)

In [ ]:
def upload_mp3(file_path):
    with open(file_path, 'rb') as mp3_file:
        file_id = fs.put(mp3_file, filename='aud.mp3')
        print(f'File uploaded with ID: {file_id}')
        return file_id

file_id = upload_mp3('preview_audio.mp3')

In [ ]:
def get_audio(file_id):
    file = fs.get(file_id)
    return file.read()

In [ ]:
audio_data = get_audio(file_id=file_id)

In [ ]:
from pydub import AudioSegment
import simpleaudio as sa
from io import BytesIO

def play_audio(audio_data):
    audio = AudioSegment.from_file(BytesIO(audio_data), format="mp3")

    play_obj = sa.play_buffer(audio.raw_data, num_channels=audio.channels,
                              bytes_per_sample=audio.sample_width,
                              sample_rate=audio.frame_rate)
    play_obj.wait_done()


play_audio(audio_data)


In [ ]:
def list_files():
    files = fs.find()
    for file in files:
        print(f'File ID: {file._id}, Filename: {file.filename}')

list_files()